# XGBoost

https://www.kaggle.com/code/cdeotte/catboost-starter-lb-0-60?scriptVersionId=158772898

In [1]:
DATA_PREPARATION_VOTE_METHOD = "max_vote_window" # "max_vote_window" or "sum_and_normalize". Decides how to aggregate the predictions of the overlapping windows

In [3]:
import os
import sys
import warnings
import gc
import pathlib

if bool(os.environ.get("KAGGLE_URL_BASE", "")):
  import sys
  # running on kaggle
  sys.path.insert(0, "/kaggle/input/hsm-source-files")
else:
  # running locally
  sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "..", "..", "..")))

import torch.nn as nn
import pandas as pd
import numpy as np
import xgboost as xgb

import torch

from src.utils.utils import get_raw_data_dir, get_processed_data_dir, get_submission_csv_path, set_seeds, get_models_save_path
from src.utils.constants import Constants
from src.datasets.eeg_processor import EEGDataProcessor
from src.utils.k_folds_creator import KFoldCreator

from tqdm import tqdm

set_seeds(42)

2025-10-07 20:02:54,634 :: root :: INFO :: Initialising Utils
2025-10-07 20:02:54,635 :: root :: INFO :: Initialising Datasets


In [4]:
DATA_PATH = get_raw_data_dir()

processor = EEGDataProcessor(raw_data_path=DATA_PATH, processed_data_path=get_processed_data_dir())
train_df = processor.process_data(vote_method=DATA_PREPARATION_VOTE_METHOD, skip_npy=True)

test_df = pd.read_csv(DATA_PATH / "test.csv")

kl_score = nn.KLDivLoss(reduction="batchmean")

Processor initialized.
Raw data path: '/home/david/git/aicomp/data'
Processed data path: '/home/david/git/aicomp/data/processed'
Starting EEG Data Processing Pipeline
Skipping NumPy file creation as requested.
Using 'max_vote_window' vote aggregation strategy.

Processed train data saved to '/home/david/git/aicomp/data/processed/train_processed.csv'.
Shape of the final dataframe: (17089, 15)

Pipeline finished successfully!


## Feature Engineering

We need features for the XGBoost model.
For this, we take the mean over time for all of the 400 spectrogram frequencies.
We take the middle 10 minutes of all spectrograms.
For each EEG ID, this produces 400 features.

## Load Spectrogram Files into Memory

In [4]:
spectrograms_dir = DATA_PATH / "train_spectrograms"
spectrogram_files = list(spectrograms_dir.glob("*.parquet"))
print(f"Found {len(spectrogram_files)} train spectrogram files to load into memory")

def get_spectrogram_content(spectrogram_file: pathlib.Path):
  spectrogram_id = int(spectrogram_file.stem.split("_")[-1])
  content = pd.read_parquet(file)
  content = content.drop(columns=["time"]).values
  return spectrogram_id, content

spectrograms = {}
for file in tqdm(spectrogram_files):
  spectrogram_id, content = get_spectrogram_content(file)
  spectrograms[spectrogram_id] = content

gc.collect()
print("Loaded all train spectrograms into memory")

Found 11138 spectrogram files to load into memory


100%|██████████| 11138/11138 [06:21<00:00, 29.17it/s]


Loaded all train spectrograms into memory


In [5]:
FEATURES = [f"spec_mean_freq_{x}" for x in range(400)]
data = np.zeros((len(train_df), len(FEATURES)))

def extract_train_spectrogram_features(row, all_spectrograms):
  spectrogram_id = int(row["spectrogram_id"])
  middle_offset = (row["min_offset"] + row["max_offset"]) // 2 # this the middle between the least spectrogram offset and greatest spectogram offset
  row_index = int(middle_offset // 2) # each spectrogram row corresponds to 2s, so we divide by 2 to get the row index
  average_frequencies = np.array(all_spectrograms[spectrogram_id][row_index:row_index+300,:] ).mean(axis=0) # average over 300 rows (10 minutes)
  return average_frequencies

for i in tqdm(range(len(train_df)), total=len(train_df)):
  row = train_df.iloc[i]
  data[i,:] = extract_train_spectrogram_features(row, spectrograms)

100%|██████████| 17089/17089 [00:02<00:00, 6249.52it/s]


In [6]:
warnings.filterwarnings("ignore", category=pd.errors.PerformanceWarning)
train_df[FEATURES] = data

del data
del spectrograms
gc.collect()

train_df.head()

,eeg_id,spectrogram_id,min_offset,max_offset,patient_id,expert_consensus,seizure_vote,lpd_vote,gpd_vote,lrda_vote,grda_vote,other_vote,spec_mean_freq_0,spec_mean_freq_1,spec_mean_freq_2,spec_mean_freq_3,spec_mean_freq_4,spec_mean_freq_5,spec_mean_freq_6,spec_mean_freq_7,spec_mean_freq_8,spec_mean_freq_9,spec_mean_freq_10,spec_mean_freq_11,spec_mean_freq_12,spec_mean_freq_13,spec_mean_freq_14,spec_mean_freq_15,spec_mean_freq_16,spec_mean_freq_17,spec_mean_freq_18,spec_mean_freq_19,spec_mean_freq_20,spec_mean_freq_21,spec_mean_freq_22,spec_mean_freq_23,spec_mean_freq_24,spec_mean_freq_25,spec_mean_freq_26,spec_mean_freq_27,spec_mean_freq_28,spec_mean_freq_29,spec_mean_freq_30,spec_mean_freq_31,spec_mean_freq_32,spec_mean_freq_33,spec_mean_freq_34,spec_mean_freq_35,spec_mean_freq_36,spec_mean_freq_37,spec_mean_freq_38,spec_mean_freq_39,spec_mean_freq_40,spec_mean_freq_41,spec_mean_freq_42,spec_mean_freq_43,spec_mean_freq_44,spec_mean_freq_45,spec_mean_freq_46,spec_mean_freq_47,spec_mean_freq_48,spec_mean_freq_49,spec_mean_freq_50,spec_mean_freq_51,spec_mean_freq_52,spec_mean_freq_53,spec_mean_freq_54,spec_mean_freq_55,spec_mean_freq_56,spec_mean_freq_57,spec_mean_freq_58,spec_mean_freq_59,spec_mean_freq_60,spec_mean_freq_61,spec_mean_freq_62,spec_mean_freq_63,spec_mean_freq_64,spec_mean_freq_65,spec_mean_freq_66,spec_mean_freq_67,spec_mean_freq_68,spec_mean_freq_69,spec_mean_freq_70,spec_mean_freq_71,spec_mean_freq_72,spec_mean_freq_73,spec_mean_freq_74,spec_mean_freq_75,spec_mean_freq_76,spec_mean_freq_77,spec_mean_freq_78,spec_mean_freq_79,spec_mean_freq_80,spec_mean_freq_81,spec_mean_freq_82,spec_mean_freq_83,spec_mean_freq_84,spec_mean_freq_85,spec_mean_freq_86,spec_mean_freq_87,spec_mean_freq_88,spec_mean_freq_89,spec_mean_freq_90,spec_mean_freq_91,spec_mean_freq_92,spec_mean_freq_93,spec_mean_freq_94,spec_mean_freq_95,spec_mean_freq_96,spec_mean_freq_97,spec_mean_freq_98,spec_mean_freq_99,spec_mean_freq_100,spec_mean_freq_101,spec_mean_freq_102,spec_mean_freq_103,spec_mean_freq_104,spec_mean_freq_105,spec_mean_freq_106,spec_mean_freq_107,spec_mean_freq_108,spec_mean_freq_109,spec_mean_freq_110,spec_mean_freq_111,spec_mean_freq_112,spec_mean_freq_113,spec_mean_freq_114,spec_mean_freq_115,spec_mean_freq_116,spec_mean_freq_117,spec_mean_freq_118,spec_mean_freq_119,spec_mean_freq_120,spec_mean_freq_121,spec_mean_freq_122,spec_mean_freq_123,spec_mean_freq_124,spec_mean_freq_125,spec_mean_freq_126,spec_mean_freq_127,spec_mean_freq_128,spec_mean_freq_129,spec_mean_freq_130,spec_mean_freq_131,spec_mean_freq_132,spec_mean_freq_133,spec_mean_freq_134,spec_mean_freq_135,spec_mean_freq_136,spec_mean_freq_137,spec_mean_freq_138,spec_mean_freq_139,spec_mean_freq_140,spec_mean_freq_141,spec_mean_freq_142,spec_mean_freq_143,spec_mean_freq_144,spec_mean_freq_145,spec_mean_freq_146,spec_mean_freq_147,spec_mean_freq_148,spec_mean_freq_149,spec_mean_freq_150,spec_mean_freq_151,spec_mean_freq_152,spec_mean_freq_153,spec_mean_freq_154,spec_mean_freq_155,spec_mean_freq_156,spec_mean_freq_157,spec_mean_freq_158,spec_mean_freq_159,spec_mean_freq_160,spec_mean_freq_161,spec_mean_freq_162,spec_mean_freq_163,spec_mean_freq_164,spec_mean_freq_165,spec_mean_freq_166,spec_mean_freq_167,spec_mean_freq_168,spec_mean_freq_169,spec_mean_freq_170,spec_mean_freq_171,spec_mean_freq_172,spec_mean_freq_173,spec_mean_freq_174,spec_mean_freq_175,spec_mean_freq_176,spec_mean_freq_177,spec_mean_freq_178,spec_mean_freq_179,spec_mean_freq_180,spec_mean_freq_181,spec_mean_freq_182,spec_mean_freq_183,spec_mean_freq_184,spec_mean_freq_185,spec_mean_freq_186,spec_mean_freq_187,spec_mean_freq_188,spec_mean_freq_189,spec_mean_freq_190,spec_mean_freq_191,spec_mean_freq_192,spec_mean_freq_193,spec_mean_freq_194,spec_mean_freq_195,spec_mean_freq_196,spec_mean_freq_197,spec_mean_freq_198,spec_mean_freq_199,spec_mean_freq_200,spec_mean_freq_201,spec_mean_freq_202,spec_mean_freq_203,spec_mean_freq_204,spec_mean_freq_205,spec_mean_freq_206,spec_mean_freq_207,spec_mean_freq_208,spec

## Train XGBoost Model

In [7]:
N_SPLITS = 5

In [8]:
fold_creator = KFoldCreator(n_splits=N_SPLITS, seed=Constants.SEED)
train_folds_df = fold_creator.create_folds(
    df=train_df, stratify_col="expert_consensus", group_col="patient_id"
)

In [9]:
all_oof = []
all_true = []
targets_dict = {"Seizure":0, "LPD":1, "GPD":2, "LRDA":3, "GRDA":4, "Other":5}

models_save_path = get_models_save_path() / "xgboost" / "spectrogram_means" / DATA_PREPARATION_VOTE_METHOD
models_save_path.mkdir(parents=True, exist_ok=True)

for fold in range(N_SPLITS):
    fold_train_df = train_folds_df[train_folds_df["fold"] != fold].reset_index(drop=True)
    fold_valid_df = train_folds_df[train_folds_df["fold"] == fold].reset_index(drop=True)

    print("=" * 40)
    print(f"FOLD {fold}")
    print(f"Train size: {len(fold_train_df)}, Valid size: {len(fold_valid_df)}")
    print("=" * 30)

    X_train = fold_train_df[FEATURES]
    y_train = fold_train_df["expert_consensus"].map(targets_dict)
    
    X_valid = fold_valid_df[FEATURES]
    y_valid = fold_valid_df["expert_consensus"].map(targets_dict)

    dtrain = xgb.DMatrix(X_train, label=y_train)
    dvalid = xgb.DMatrix(X_valid, label=y_valid)

    params = {
        "objective": "multi:softprob",
        "num_class": len(Constants.TARGETS),
        "device": "cuda",
        "tree_method": "hist",
        "eval_metric": "mlogloss",
        "seed": Constants.SEED,
    }

    evals = [(dvalid, "eval")]
    model = xgb.train(
        params,
        dtrain,
        num_boost_round=300,
        evals=evals,
        verbose_eval=100,
        early_stopping_rounds=10,
    )
    
    model.save_model(models_save_path / f"fold_{fold}.json")

    oof = model.predict(dvalid)
    all_oof.extend(oof)

    all_true.extend(fold_valid_df[Constants.TARGETS].values)

    del X_train, y_train, X_valid, y_valid, dtrain, dvalid, oof
    gc.collect()

all_oof = np.array(all_oof)
all_true = np.array(all_true)

FOLD 0
Train size: 13755, Valid size: 3334
[0]	eval-mlogloss:1.63018
[26]	eval-mlogloss:1.35269
FOLD 1
Train size: 13151, Valid size: 3938
[0]	eval-mlogloss:1.60886
[27]	eval-mlogloss:1.32410
FOLD 2
Train size: 13422, Valid size: 3667
[0]	eval-mlogloss:1.62823
[22]	eval-mlogloss:1.32224
FOLD 3
Train size: 14356, Valid size: 2733
[0]	eval-mlogloss:1.61536
[32]	eval-mlogloss:1.23476
FOLD 4
Train size: 13672, Valid size: 3417
[0]	eval-mlogloss:1.64184
[19]	eval-mlogloss:1.41802


## CV Score

In [31]:
all_oof_tensor = torch.tensor(all_oof, dtype=torch.float32)
all_true_tensor = torch.tensor(all_true, dtype=torch.float32)

kl_score = nn.KLDivLoss(reduction="batchmean")
score = kl_score(all_oof_tensor.log(), all_true_tensor).item()

print(f"OOF KL Score: {score}")

OOF KL Score: 1.055870532989502


## Infer on Test and create Submission

In [11]:
del train_df
gc.collect()

0

In [13]:
test_spectrograms_dir = DATA_PATH / "test_spectrograms"
test_spectrogram_files = list(test_spectrograms_dir.glob("*.parquet"))
print(f"Found {len(test_spectrogram_files)} test spectrogram files to load into memory")

test_spectrograms = {}
for file in tqdm(test_spectrogram_files):
  spectrogram_id, content = get_spectrogram_content(file)
  test_spectrograms[spectrogram_id] = content

gc.collect()
print("Loaded all test spectrograms into memory")

Found 1 test spectrogram files to load into memory


100%|██████████| 1/1 [00:00<00:00, 28.70it/s]

Loaded all test spectrograms into memory


In [23]:
test_data = np.zeros((len(test_df), len(FEATURES)))

def extract_test_spectrogram_features(row, all_spectrograms):
  # this differs from train because all test spectrograms are exactly 10 minutes long, so we don't need to extract the center window
  spectrogram_id = int(row["spectrogram_id"])
  average_frequencies = np.array(all_spectrograms[spectrogram_id][:] ).mean(axis=0) # average over 300 rows (10 minutes)
  return average_frequencies

for i in tqdm(range(len(test_df)), total=len(test_df)):
  row = test_df.iloc[i]
  test_data[i,:] = extract_test_spectrogram_features(row, test_spectrograms)

100%|██████████| 1/1 [00:00<00:00, 1406.07it/s]


In [24]:
test_df[FEATURES] = test_data

del test_data
del test_spectrograms
gc.collect()

test_df.head()

,spectrogram_id,eeg_id,patient_id,spec_mean_freq_0,spec_mean_freq_1,spec_mean_freq_2,spec_mean_freq_3,spec_mean_freq_4,spec_mean_freq_5,spec_mean_freq_6,spec_mean_freq_7,spec_mean_freq_8,spec_mean_freq_9,spec_mean_freq_10,spec_mean_freq_11,spec_mean_freq_12,spec_mean_freq_13,spec_mean_freq_14,spec_mean_freq_15,spec_mean_freq_16,spec_mean_freq_17,spec_mean_freq_18,spec_mean_freq_19,spec_mean_freq_20,spec_mean_freq_21,spec_mean_freq_22,spec_mean_freq_23,spec_mean_freq_24,spec_mean_freq_25,spec_mean_freq_26,spec_mean_freq_27,spec_mean_freq_28,spec_mean_freq_29,spec_mean_freq_30,spec_mean_freq_31,spec_mean_freq_32,spec_mean_freq_33,spec_mean_freq_34,spec_mean_freq_35,spec_mean_freq_36,spec_mean_freq_37,spec_mean_freq_38,spec_mean_freq_39,spec_mean_freq_40,spec_mean_freq_41,spec_mean_freq_42,spec_mean_freq_43,spec_mean_freq_44,spec_mean_freq_45,spec_mean_freq_46,spec_mean_freq_47,spec_mean_freq_48,spec_mean_freq_49,spec_mean_freq_50,spec_mean_freq_51,spec_mean_freq_52,spec_mean_freq_53,spec_mean_freq_54,spec_mean_freq_55,spec_mean_freq_56,spec_mean_freq_57,spec_mean_freq_58,spec_mean_freq_59,spec_mean_freq_60,spec_mean_freq_61,spec_mean_freq_62,spec_mean_freq_63,spec_mean_freq_64,spec_mean_freq_65,spec_mean_freq_66,spec_mean_freq_67,spec_mean_freq_68,spec_mean_freq_69,spec_mean_freq_70,spec_mean_freq_71,spec_mean_freq_72,spec_mean_freq_73,spec_mean_freq_74,spec_mean_freq_75,spec_mean_freq_76,spec_mean_freq_77,spec_mean_freq_78,spec_mean_freq_79,spec_mean_freq_80,spec_mean_freq_81,spec_mean_freq_82,spec_mean_freq_83,spec_mean_freq_84,spec_mean_freq_85,spec_mean_freq_86,spec_mean_freq_87,spec_mean_freq_88,spec_mean_freq_89,spec_mean_freq_90,spec_mean_freq_91,spec_mean_freq_92,spec_mean_freq_93,spec_mean_freq_94,spec_mean_freq_95,spec_mean_freq_96,spec_mean_freq_97,spec_mean_freq_98,spec_mean_freq_99,spec_mean_freq_100,spec_mean_freq_101,spec_mean_freq_102,spec_mean_freq_103,spec_mean_freq_104,spec_mean_freq_105,spec_mean_freq_106,spec_mean_freq_107,spec_mean_freq_108,spec_mean_freq_109,spec_mean_freq_110,spec_mean_freq_111,spec_mean_freq_112,spec_mean_freq_113,spec_mean_freq_114,spec_mean_freq_115,spec_mean_freq_116,spec_mean_freq_117,spec_mean_freq_118,spec_mean_freq_119,spec_mean_freq_120,spec_mean_freq_121,spec_mean_freq_122,spec_mean_freq_123,spec_mean_freq_124,spec_mean_freq_125,spec_mean_freq_126,spec_mean_freq_127,spec_mean_freq_128,spec_mean_freq_129,spec_mean_freq_130,spec_mean_freq_131,spec_mean_freq_132,spec_mean_freq_133,spec_mean_freq_134,spec_mean_freq_135,spec_mean_freq_136,spec_mean_freq_137,spec_mean_freq_138,spec_mean_freq_139,spec_mean_freq_140,spec_mean_freq_141,spec_mean_freq_142,spec_mean_freq_143,spec_mean_freq_144,spec_mean_freq_145,spec_mean_freq_146,spec_mean_freq_147,spec_mean_freq_148,spec_mean_freq_149,spec_mean_freq_150,spec_mean_freq_151,spec_mean_freq_152,spec_mean_freq_153,spec_mean_freq_154,spec_mean_freq_155,spec_mean_freq_156,spec_mean_freq_157,spec_mean_freq_158,spec_mean_freq_159,spec_mean_freq_160,spec_mean_freq_161,spec_mean_freq_162,spec_mean_freq_163,spec_mean_freq_164,spec_mean_freq_165,spec_mean_freq_166,spec_mean_freq_167,spec_mean_freq_168,spec_mean_freq_169,spec_mean_freq_170,spec_mean_freq_171,spec_mean_freq_172,spec_mean_freq_173,spec_mean_freq_174,spec_mean_freq_175,spec_mean_freq_176,spec_mean_freq_177,spec_mean_freq_178,spec_mean_freq_179,spec_mean_freq_180,spec_mean_freq_181,spec_mean_freq_182,spec_mean_freq_183,spec_mean_freq_184,spec_mean_freq_185,spec_mean_freq_186,spec_mean_freq_187,spec_mean_freq_188,spec_mean_freq_189,spec_mean_freq_190,spec_mean_freq_191,spec_mean_freq_192,spec_mean_freq_193,spec_mean_freq_194,spec_mean_freq_195,spec_mean_freq_196,spec_mean_freq_197,spec_mean_freq_198,spec_mean_freq_199,spec_mean_freq_200,spec_mean_freq_201,spec_mean_freq_202,spec_mean_freq_203,spec_mean_freq_204,spec_mean_freq_205,spec_mean_freq_206,spec_mean_freq_207,spec_mean_freq_208,spec_mean_freq_209,spec_mean_freq_210,spec_mean_freq_211,spec_mean_freq_212,spec_mean_freq_213,spec_mean_

In [27]:
test_preds = []

for fold in range(N_SPLITS):
  print("=" * 40)
  print(f"Predicting fold {fold}")
  print("=" * 40)

  X_train = test_df[FEATURES]
  dtest = xgb.DMatrix(X_train)

  model = xgb.Booster()
  model.load_model(models_save_path / f"fold_{fold}.json")

  preds = model.predict(dtest)
  test_preds.append(preds)

test_preds = np.mean(test_preds, axis=0)
print(f"Test predictions shape: {test_preds.shape}")

Predicting fold 0
Predicting fold 1
Predicting fold 2
Predicting fold 3
Predicting fold 4
Test predictions shape: (1, 6)


In [29]:
# sanity check: all predictions should sum to 1
assert np.allclose(test_preds.sum(axis=1), 1.0)

In [30]:
submission = pd.DataFrame({"eeg_id": test_df["eeg_id"]})
submission[Constants.TARGETS] = test_preds

submission.to_csv(get_submission_csv_path(), index=False)